[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1sJlKYh8-LUl_1txX72F-kfmSMtNMcgHc?usp=sharing) 


In [1]:
import pandas as pd

# Lista de nombres de archivos CSV incluyendo los primeros dos archivos
file_names = [ "outputvision-missing_fotos.csv",
    "outputvision-1-250.csv", "outputvision-251-500.csv", "outputvision-501-750.csv",
    "outputvision-751-1000.csv", "outputvision-1001-1250.csv", "outputvision-1250-1750.csv",
    "outputvision-1750-2000.csv", "outputvision-2001-2250.csv", "outputvision-2251-2500.csv",
    "outputvision-2500-2750.csv", "outputvision-2751-2937.csv"
]

# Leer todos los archivos y almacenarlos en una lista de dataframes
dataframes = [pd.read_csv(file) for file in file_names]

# Combinar todos los dataframes en uno solo
df_combined = pd.concat(dataframes, ignore_index=True)

# Guardar el dataframe combinado en un nuevo archivo CSV
df_combined.to_csv("outputvision-combined.csv", index=False)


In [3]:
import pandas as pd

# Leer los archivos CSV en dataframes
df_prompt = pd.read_csv("df_prompt.csv")
df_combined = pd.read_csv("outputvision-combined.csv")

# Asegurarse de que las columnas relevantes están en formato de cadena
df_prompt['nombre_foto'] = df_prompt['nombre_foto'].astype(str)
df_combined['nombre_foto'] = df_combined['nombre_foto'].astype(str)

# Encontrar los valores en 'nombre_foto' de df_prompt que no están en df_combined
missing_fotos = df_prompt[~df_prompt['nombre_foto'].isin(df_combined['nombre_foto'])]

# Guardar los resultados en un nuevo archivo CSV si es necesario
missing_fotos.to_csv("missing_fotos.csv", index=False)

# Mostrar los resultados
print(missing_fotos)


Empty DataFrame
Columns: [x, y, parte_id, fecha, hora, rango, clasificacion_parte, source, source_glosa, estado, lugar, grupo_delictual, delito, riesgo_situacional, descripcion_situacional, denuncia_formal, denuncia_informa, comuna, id_comuna, barrio, id_barrio, geometry, nearest_node, nombre_foto]
Index: []

[0 rows x 24 columns]


In [2]:
import pandas as pd 
df_combined = pd.read_csv("outputvision-combined.csv")


In [3]:
df_combined

,Unnamed: 0,x,y,parte_id,fecha,hora,rango,clasificacion_parte,source,source_glosa,...,denuncia_formal,denuncia_informa,comuna,id_comuna,barrio,id_barrio,geometry,nearest_node,nombre_foto,text
0,0,-70.63623,-33.43079,349887,2015-01-02,23:05:00,20:00-23:59,Detención,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63623 -33.43079),2.459727e+09,"imagenes_stgo/-33.42932592,-70.63532995535715.jpg",The image shows a street viewed from a high va...
1,1,-70.63018,-33.44323,586642,2014-02-21,10:10:00,8:00-11:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63018 -33.44323),1.337689e+08,"imagenes_stgo/-33.44341660839161,-70.630142264...",The image shows a street with two parked cars ...
2,2,-70.62536,-33.42980,42584,2015-06-20,01:00:00,0:00-3:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.62536 -33.4298),2.244146e+08,"imagenes_stgo/-33.429514599369085,-70.62501985...",The image shows a street with a white car park...
3,3,-70.62135,-33.42979,7211180,2018-05-02,12:00:00,12:00-15:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.62135 -33.42979),7.042676e+08,"imagenes_stgo/-33.429964215999995,-70.62090071...","The image shows a street with a clear sky, ind..."
4,4,-70.61827,-33.41261,3049817,2015-09-03,18:45:00,16:00-19:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.61827 -33.41261),2.656867e+08,"imagenes_stgo/-33.41154513564668,-70.616374813...","The image shows a street lined with trees, ind..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181,1855143,-70.58418,-33.43388,3453012,2016-04-26,20:35:00,20:00-23:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58418 -33.43388),6.051849e+08,"imagenes_stgo/-33.433796271999995,-70.58413650...",The image shows a street with a paved road tha...
3182,1855406,-70.58376,-33.43535,3500747,2016-06-15,20:00:00,20:00-23:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58376 -33.43535),9.040377e+08,"imagenes_stgo/-33.43505852229299,-70.583428284...",The image shows a residential street with a wh...
3183,1855408,-70.58374,-33.43337,4848048,2017-08-11,10:00:00,8:00-11:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58374 -33.43337),2.540342e+08,"imagenes_stgo/-33.43336751048951,-70.5834295.jpg",The image shows a street with a mix of residen...
3184,1855460,-70.58335,-33.43400,3793505,2016-03-30,06:00:00,4:00-7:59,Denuncia,caso_policial,Caso policial,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58335 -33.434),2.648450e+08,"imagenes_stgo/-33.43400696529968,-70.583029643...",The image shows a residential street with a si...


In [9]:
import pandas as pd

condition_length = df_combined['text'].str.len() < 50

# Filtrar las filas donde las cadenas contienen los caracteres '<', '>', o '|'
condition_chars = df_combined['text'].str.contains(r'[<>|]')

# Usar una condición 'o' para filtrar las filas que cumplen al menos una de las condiciones
filtered_df = df_combined[condition_length | condition_chars]
filtered_df.to_csv('errores.csv')


In [11]:
filtered_df_sin_errores  = pd.read_csv('output-vision-errores.csv')

# Paso 1: Identificar las filas a eliminar (basado en el índice)
indices_a_eliminar = filtered_df.index

# Paso 2: Eliminar las filas de df_combined que están en filtered_df
df_combined_sin_errores = df_combined.drop(indices_a_eliminar)

# Paso 3: Agregar las filas del nuevo DataFrame filtered_df_sin_errores a df_combined
df_combined_actualizado = pd.concat([df_combined_sin_errores, filtered_df_sin_errores])

# Resetear el índice si es necesario
df_combined_actualizado.reset_index(drop=True, inplace=True)

# Mostrar el DataFrame actualizado
print(df_combined_actualizado)

      Unnamed: 0         x         y  parte_id       fecha      hora  \
0              0 -70.63623 -33.43079    349887  2015-01-02  23:05:00   
1              1 -70.63018 -33.44323    586642  2014-02-21  10:10:00   
2              2 -70.62536 -33.42980     42584  2015-06-20  01:00:00   
3              3 -70.62135 -33.42979   7211180  2018-05-02  12:00:00   
4              4 -70.61827 -33.41261   3049817  2015-09-03  18:45:00   
...          ...       ...       ...       ...         ...       ...   
3181     1855474 -70.58321 -33.43365    646478  2013-02-11  09:00:00   
3182       82082 -70.63252 -33.43486    102396  2014-03-01  23:50:00   
3183      728486 -70.61827 -33.41261   3049817  2015-09-03  18:45:00   
3184      801889 -70.61369 -33.40972   5622617  2017-06-29  18:30:00   
3185     1059207 -70.61260 -33.41921   1985118  2014-08-08  08:30:00   

            rango clasificacion_parte         source   source_glosa  ...  \
0     20:00-23:59           Detención  caso_policial  Caso 

In [14]:

condition_length = df_combined_actualizado['text'].str.len() < 50

# Filtrar las filas donde las cadenas contienen los caracteres '<', '>', o '|'
condition_chars = df_combined_actualizado['text'].str.contains(r'[<>|]')

# Usar una condición 'o' para filtrar las filas que cumplen al menos una de las condiciones
filtered_df = df_combined_actualizado[condition_length | condition_chars]
filtered_df

,Unnamed: 0,x,y,parte_id,fecha,hora,rango,clasificacion_parte,source,source_glosa,...,comuna,id_comuna,barrio,id_barrio,geometry,nearest_node,nombre_foto,text,Unnamed: 0.2,Unnamed: 0.1


In [15]:
delitos = pd.read_csv('delitos_imagenes_providencia_traducido.csv')

C:\Users\benja\AppData\Local\Temp\ipykernel_10712\2564423022.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  delitos = pd.read_csv('delitos_imagenes_providencia_traducido.csv')


In [16]:
delitos_merged = pd.merge(delitos, df_combined_actualizado[['nombre_foto', 'text']], on='nombre_foto', how='left')


In [17]:
# Cuenta los NaN en la columna nombre_foto
na_nombre_foto = delitos_merged['nombre_foto'].isna().sum()

# Cuenta los NaN en la columna text
na_text = delitos_merged['text'].isna().sum()

print(f"Cantidad de NaN en 'nombre_foto': {na_nombre_foto}")
print(f"Cantidad de NaN en 'text': {na_text}")


Cantidad de NaN en 'nombre_foto': 1250816
Cantidad de NaN en 'text': 1250816


In [18]:
# Cuenta los valores no NaN en la columna nombre_foto
not_na_nombre_foto = delitos_merged['nombre_foto'].notna().sum()

# Cuenta los valores no NaN en la columna text
not_na_text = delitos_merged['text'].notna().sum()

print(f"Cantidad de valores no NaN en 'nombre_foto': {not_na_nombre_foto}")
print(f"Cantidad de valores no NaN en 'text': {not_na_text}")


Cantidad de valores no NaN en 'nombre_foto': 651559
Cantidad de valores no NaN en 'text': 651559


In [19]:
# Contar el número de columnas usando len() en columns
num_columnas = len(delitos_merged.columns)
print(f"Número de columnas: {num_columnas}")


Número de columnas: 25


In [20]:
num_columnas = delitos_merged.shape[0]
print(f"Número de columnas: {num_columnas}")


Número de columnas: 1902375


In [21]:
print(delitos_merged.columns)

Index(['x', 'y', 'parte_id', 'fecha', 'hora', 'rango', 'clasificacion_parte',
       'source', 'source_glosa', 'estado', 'lugar', 'grupo_delictual',
       'delito', 'riesgo_situacional', 'descripcion_situacional',
       'denuncia_formal', 'denuncia_informa', 'comuna', 'id_comuna', 'barrio',
       'id_barrio', 'geometry', 'nearest_node', 'nombre_foto', 'text'],
      dtype='object')


In [22]:
delitos_merged.to_csv('delitos_txt.csv')

In [23]:
delitos_merged

,x,y,parte_id,fecha,hora,rango,clasificacion_parte,source,source_glosa,estado,...,denuncia_formal,denuncia_informa,comuna,id_comuna,barrio,id_barrio,geometry,nearest_node,nombre_foto,text
0,-70.63627,-33.43064,5733886,2017-06-26,00:20:00,0:00-3:59,Report,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
1,-70.63627,-33.43064,349882,2015-02-12,02:10:00,0:00-3:59,NaN,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
2,-70.63627,-33.43064,349884,2015-02-21,01:20:00,0:00-3:59,NaN,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
3,-70.63627,-33.43064,349890,2015-06-23,09:05:00,8:00-11:59,Report,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
4,-70.63627,-33.43064,349891,2015-03-04,02:45:00,0:00-3:59,Report,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902370,-70.58291,-33.43364,3343563,2013-04-12,20:40:00,20:00-23:59,Report,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58291 -33.43364),264844969.0,"imagenes_stgo/-33.433796271999995,-70.58272249...",The image shows a street with a paved road and...
1902371,-70.58291,-33.43364,3735312,2013-01-11,10:30:00,8:00-11:59,Report,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58291 -33.43364),264844969.0,"imagenes_stgo/-33.433796271999995,-70.58272249...",The image shows a street with a paved road and...
1902372,-70.58281,-33.43429,180049,2013-01-23,19:30:00,16:00-19:59,Report,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58281 -33.43429),264844968.0,"imagenes_stgo/-33.43400696529968,-70.583029643...",The image shows a residential street with a si...
1902373,-70.58256,-33.43400,450812,2013-01-28,18:43:00,16:00-19:59,Report,police_case,Police case,NaN,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58256 -33.434),254034223.0,NaN,NaN


In [24]:
import pandas as pd 
delitos_merged = pd.read_csv('delitos_txt.csv')

C:\Users\benja\AppData\Local\Temp\ipykernel_10712\1446381070.py:2: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  delitos_merged = pd.read_csv('delitos_txt.csv')


In [25]:
delitos_merged

,Unnamed: 0,x,y,parte_id,fecha,hora,rango,clasificacion_parte,source,source_glosa,...,denuncia_formal,denuncia_informa,comuna,id_comuna,barrio,id_barrio,geometry,nearest_node,nombre_foto,text
0,0,-70.63627,-33.43064,5733886,2017-06-26,00:20:00,0:00-3:59,Report,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
1,1,-70.63627,-33.43064,349882,2015-02-12,02:10:00,0:00-3:59,NaN,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
2,2,-70.63627,-33.43064,349884,2015-02-21,01:20:00,0:00-3:59,NaN,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
3,3,-70.63627,-33.43064,349890,2015-06-23,09:05:00,8:00-11:59,Report,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
4,4,-70.63627,-33.43064,349891,2015-03-04,02:45:00,0:00-3:59,Report,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.63627 -33.43064),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902370,1902370,-70.58291,-33.43364,3343563,2013-04-12,20:40:00,20:00-23:59,Report,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58291 -33.43364),264844969.0,"imagenes_stgo/-33.433796271999995,-70.58272249...",The image shows a street with a paved road and...
1902371,1902371,-70.58291,-33.43364,3735312,2013-01-11,10:30:00,8:00-11:59,Report,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58291 -33.43364),264844969.0,"imagenes_stgo/-33.433796271999995,-70.58272249...",The image shows a street with a paved road and...
1902372,1902372,-70.58281,-33.43429,180049,2013-01-23,19:30:00,16:00-19:59,Report,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58281 -33.43429),264844968.0,"imagenes_stgo/-33.43400696529968,-70.583029643...",The image shows a residential street with a si...
1902373,1902373,-70.58256,-33.43400,450812,2013-01-28,18:43:00,16:00-19:59,Report,police_case,Police case,...,NaN,NaN,Providencia,13123,NaN,NaN,POINT (-70.58256 -33.434),254034223.0,NaN,NaN


In [26]:
df = delitos_merged.loc[:, ['clasificacion_parte', 'source_glosa', 'lugar', 'grupo_delictual', 'delito', 'riesgo_situacional', 'descripcion_situacional', 'denuncia_formal', 'denuncia_informa', 'text']]


In [27]:
df = df.astype(str)

# Unir las columnas en una única columna llamada 'txt'
df['txt'] = df.apply(lambda row: ' '.join(x for x in row if x != 'nan'), axis=1)

# Guardar todas las filas en un archivo de texto
df['txt'].to_csv('data_con_txt.txt', index=False, header=False)

# Guardar el DataFrame con la nueva columna 'txt' en un archivo CSV
df.to_csv('data_con_txt.csv', index=False)

In [28]:
# Especifica el número de líneas que deseas ver
num_lines = 5

with open('data_con_txt.txt', 'r') as file:
    for _ in range(num_lines):
        print(file.readline().strip())


Report Police case Public road Incivilities Drunkenness
Police case Public road Other Crimes and Offenses Arrest warrant
Police case Public road Other Crimes and Offenses Arrest warrant
Report Police case Public road Incivilities Damages
Report Police case Public road Crimes with High Social Impact against Persons Robbery with violence or intimidation
